In [1]:
#Question 1
import datetime

def has_friday_13th(month, year):
    # Validate month
    if not (1 <= month <= 12):
        raise ValueError("Month must be between 1 and 12")
    
    # Validate year (assuming any reasonable year is acceptable)
    if not (1000 <= year <= 9999):
        raise ValueError("Year must be a four-digit number")
    
    # Create a date for the 13th of the specified month and year
    date_to_check = datetime.date(year, month, 13)
    
    # Check if the 13th is a Friday
    return date_to_check.weekday() == 4  # weekday() returns 4 for Friday

# Example usage:
print(has_friday_13th(5, 2004))  # Output: False (May 2004 does not have a Friday the 13th)
print(has_friday_13th(3, 2020))  # Output: True (March 2020 has a Friday the 13th)

False
True


In [2]:
# Question 2
def parse_encoded_string(encoded_str):
    """
    Parses an encoded string containing the first name, second name, and ID.

    Parameters:
    encoded_str (str): The encoded string in the format "first_namesecond_nameid".

    Returns:
    tuple: A tuple containing the first name, second name, and ID.
    """
    # Find the first occurrence of '000' that signifies the end of the first name
    first_name_end = encoded_str.find('000')
    if first_name_end == -1:
        raise ValueError("The encoded string does not contain a proper '000' separator for the first name.")

    # Extract the first name
    first_name = encoded_str[:first_name_end]

    # Find the first occurrence of '000' after the first name that signifies the end of the second name
    second_name_end = encoded_str.find('000', first_name_end + 3)
    if second_name_end == -1:
        raise ValueError("The encoded string does not contain a proper '000' separator for the second name.")

    # Extract the second name
    second_name = encoded_str[first_name_end + 3:second_name_end]

    # Extract the ID
    id = encoded_str[second_name_end + 3:]

    # Remove leading zeros from ID
    id = id.lstrip('0')

    return first_name, second_name, id

# Example usage:
encoded_str = "Robert000Smith000123"
first_name, second_name, id = parse_encoded_string(encoded_str)
print(f"First Name: {first_name}, Second Name: {second_name}, ID: {id}")

First Name: Robert, Second Name: Smith, ID: 123


In [3]:
#Question 3
import numpy as np

# Define the decision tree classifier class
class DecisionNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature  # Index of feature to split on
        self.threshold = threshold  # Threshold value for splitting
        self.left = left  # Left subtree (for values less than or equal to threshold)
        self.right = right  # Right subtree (for values greater than threshold)
        self.value = value  # Value to return if leaf node

class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        if not isinstance(y, list):
            y = y.tolist()  # Convert y to list if it's not already
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        if depth == self.max_depth or len(set(y)) == 1:
            return DecisionNode(value=max(set(y), key=y.count))

        n_samples, n_features = X.shape
        best_feature, best_threshold = self._find_best_split(X, y)

        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold
        left_node = self._build_tree(X[left_indices], [y[i] for i, x in enumerate(left_indices) if x], depth + 1)
        right_node = self._build_tree(X[right_indices], [y[i] for i, x in enumerate(right_indices) if x], depth + 1)

        return DecisionNode(feature=best_feature, threshold=best_threshold, left=left_node, right=right_node)

    def _find_best_split(self, X, y):
        best_feature, best_threshold, best_info_gain = None, None, -float('inf')

        for feature in range(X.shape[1]):
            thresholds = sorted(set(X[:, feature]))
            for threshold in thresholds:
                left_indices = X[:, feature] <= threshold
                right_indices = X[:, feature] > threshold
                info_gain = self._information_gain(y, left_indices, right_indices)
                if info_gain > best_info_gain:
                    best_feature, best_threshold, best_info_gain = feature, threshold, info_gain

        return best_feature, best_threshold

    def _entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))  # Adding small value to avoid division by zero

    def _information_gain(self, y, left_indices, right_indices):
        parent_entropy = self._entropy(y)
        left_weight = np.sum(left_indices) / len(y)
        right_weight = np.sum(right_indices) / len(y)
        children_entropy = left_weight * self._entropy([y[i] for i, x in enumerate(left_indices) if x]) + \
                           right_weight * self._entropy([y[i] for i, x in enumerate(right_indices) if x])
        return parent_entropy - children_entropy

    def predict(self, X):
        return np.array([self._predict_instance(x, self.root) for x in X])

    def _predict_instance(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self._predict_instance(x, node.left)
        else:
            return self._predict_instance(x, node.right)

    def print_tree(self):
        self._print_tree_recursive(self.root, "")

    def _print_tree_recursive(self, node, indent):
        if node is None:
            return
        if node.value is not None:
            print(indent + "Predict:", node.value)
            return
        print(indent + "Feature", node.feature, "<=", node.threshold)
        print(indent + "|- Left:", end="")
        self._print_tree_recursive(node.left, indent + "  ")
        print(indent + "|- Right:", end="")
        self._print_tree_recursive(node.right, indent + "  ")

# Define the dataset
dataset = np.array([
    [2.5, 3.4, 1.2, 0],
    [1.7, 2.3, 3.1, 0],
    [3.1, 2.9, 1.5, 1],
    [2.6, 3.1, 2.2, 1],
    [3.1, 3.6, 1.8, 0],
    [2.9, 2.4, 3.0, 1],
    [2.7, 3.2, 2.9, 0],
    [3.4, 2.7, 1.6, 1],
    [2.5, 3.5, 2.8, 0],
    [3.2, 2.8, 1.7, 1]
])

# Separate features (X) and target labels (y)
X = dataset[:, :-1]
y = dataset[:, -1]

# Train the decision tree classifier
tree_classifier = DecisionTreeClassifier(max_depth=3)
tree_classifier.fit(X, y)

# Print the decision tree
print("Decision Tree:")
tree_classifier.print_tree()

# Example usage: Predicting a new instance
new_instance = np.array([[2.8, 3.0, 1.4]])
predicted_class = tree_classifier.predict(new_instance)
print("Predicted class label for the new instance:", predicted_class[0])

Decision Tree:
Feature 1 <= 3.1
|- Left:  Feature 0 <= 1.7
  |- Left:    Predict: 0.0
  |- Right:    Predict: 1.0
|- Right:  Predict: 0.0
Predicted class label for the new instance: 1.0


In [ ]:
#Question 4
import time

class AVLNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
        self.height = 1

class AVLTree:
    def __init__(self):
        self.root = None

    def _height(self, node):
        if not node:
            return 0
        return node.height

    def _balance_factor(self, node):
        if not node:
            return 0
        return self._height(node.left) - self._height(node.right)

    def _right_rotate(self, z):
        y = z.left
        T3 = y.right

        y.right = z
        z.left = T3

        z.height = 1 + max(self._height(z.left), self._height(z.right))
        y.height = 1 + max(self._height(y.left), self._height(y.right))

        return y

    def _left_rotate(self, z):
        y = z.right
        T2 = y.left

        y.left = z
        z.right = T2

        z.height = 1 + max(self._height(z.left), self._height(z.right))
        y.height = 1 + max(self._height(y.left), self._height(y.right))

        return y

    def insert(self, root, val):
        if not root:
            return AVLNode(val)
        elif val < root.val:
            root.left = self.insert(root.left, val)
        else:
            root.right = self.insert(root.right, val)

        root.height = 1 + max(self._height(root.left), self._height(root.right))

        balance = self._balance_factor(root)

        if balance > 1:
            if val < root.left.val:
                return self._right_rotate(root)
            else:
                root.left = self._left_rotate(root.left)
                return self._right_rotate(root)

        if balance < -1:
            if val > root.right.val:
                return self._left_rotate(root)
            else:
                root.right = self._right_rotate(root.right)
                return self._left_rotate(root)

        return root

    def build_tree(self, arr):
        for val in arr:
            self.root = self.insert(self.root, val)

    def in_order_traversal(self, root):
        result = []
        if root:
            result = self.in_order_traversal(root.left)
            result.append(root.val)
            result += self.in_order_traversal(root.right)
        return result

    def search(self, root, val):
        if not root:
            return False
        if root.val == val:
            return True
        if val < root.val:
            return self.search(root.left, val)
        else:
            return self.search(root.right, val)

    def delete(self, root, val):
        if not root:
            return root

        if val < root.val:
            root.left = self.delete(root.left, val)
        elif val > root.val:
            root.right = self.delete(root.right, val)
        else:
            if not root.left:
                temp = root.right
                root = None
                return temp
            elif not root.right:
                temp = root.left
                root = None
                return temp

            temp = self._min_value_node(root.right)
            root.val = temp.val
            root.right = self.delete(root.right, temp.val)

        if not root:
            return root

        root.height = 1 + max(self._height(root.left), self._height(root.right))
        balance = self._balance_factor(root)

        if balance > 1:
            if self._balance_factor(root.left) >= 0:
                return self._right_rotate(root)
            else:
                root.left = self._left_rotate(root.left)
                return self._right_rotate(root)

        if balance < -1:
            if self._balance_factor(root.right) <= 0:
                return self._left_rotate(root)
            else:
                root.right = self._right_rotate(root.right)
                return self._left_rotate(root)

        return root

    def _min_value_node(self, node):
        current = node
        while current.left:
            current = current.left
        return current

    def performance_analysis(self, arr):
        start_time = time.time()
        self.build_tree(arr)
        end_time = time.time()
        build_time = end_time - start_time

        start_time = time.time()
        sorted_array = self.in_order_traversal(self.root)
        end_time = time.time()
        traversal_time = end_time - start_time

        print("Performance Analysis:")
        print("Build Time:", build_time, "seconds")
        print("Traversal Time:", traversal_time, "seconds")

def main():
    arr = [3, 2, 1, 4, 5, 6, 7]
    avl_tree = AVLTree()
    avl_tree.performance_analysis(arr)
    print("Initial Sorted Array:", avl_tree.in_order_traversal(avl_tree.root))

    while True:
        print("\nMenu:")
        print("1. Search")
        print("2. Delete")
        print("3. Exit")

        choice = int(input("Enter your choice: "))
        print("Choice:", choice)

        if choice == 1:
            val = int(input("Enter value to search: "))
            print("Searching for value:", val)
            if avl_tree.search(avl_tree.root, val):
                print("Value found in tree.")
            else:
                print("Value not found in tree.")
        elif choice == 2:
            val = int(input("Enter value to delete: "))
            print("Deleting value:", val)
            avl_tree.root = avl_tree.delete(avl_tree.root, val)
            print("Value deleted from tree.")
            print("Sorted Array after deletion:", avl_tree.in_order_traversal(avl_tree.root))
        elif choice == 3:
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please try again.")

    avl_tree.performance_analysis(arr)

if __name__ == "__main__":
    main()

Performance Analysis:
Build Time: 0.0 seconds
Traversal Time: 0.0 seconds
Initial Sorted Array: [1, 2, 3, 4, 5, 6, 7]

Menu:
1. Search
2. Delete
3. Exit
